In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import os, sys
import json

from run_search import find_similar_vectors

In [2]:
sys.path.append('../pubmed_rag')

Using this notebook to develop the prompt for Llama3.1. 

Requires that the DB already made.

In [7]:
vector_query = '''
what is a knowledge graph (KG)?
'''
path_config = '/users/anglup/GitHub/pubmed-rag/config.yaml'
path_to_repo = '/users/anglup/github/pubmed-rag'

In [8]:
results = find_similar_vectors(
    path_config,
    vector_query,
)
# for x in (results):
#     print(json.dumps(x, indent=4))

Path to config file: /users/anglup/GitHub/pubmed-rag/config.yaml
Loading config params ... 
Configuration: {'pmid file path': 'notebooks/data/test_pmid_list.csv', 'biocjson output path': 'output', 'condense sections': True, 'max_tokens': 200, 'transformer_model': 'NeuML/pubmedbert-base-embeddings', 'output_dimensions': 768, 'tsne': 'tsne-200t-NeuML.png', 'db name': '/users/anglup/github/pubmed-rag/demo.db', 'collection name': 't200_NeuML', 'metric_type': 'COSINE', 'n_results': 30}
Accessing /users/anglup/github/pubmed-rag/demo.db
Loading model NeuML/pubmedbert-base-embeddings from HuggingFace
Embedding query: 
what is a knowledge graph (KG)?

Searching against vector database


In [9]:
for x in (results):
    print(json.dumps(x, indent=4))

{
    "id": 1514,
    "distance": 0.7105146646499634,
    "entity": {
        "section": "intro",
        "pmid": 35150235,
        "text": "Knowledge Graph (KG) is a way to store knowledge and reveal the dynamic development law of a field . KG represents facts in the real-world through a large number of triplets (head entity, relation and tail entity), denoted as . Large integrated KGs like Freebase and DBpedia keep expanding . They have been successfully used in many applications, such as recommendation systems and question answering . In the field of biomedicine, the application of KG is becoming increasingly popular for its specialized knowledge that only domain experts can understand well . The influential roles of KG in predicting protein drug targets and adverse drug reactions are both convincing examples . The triplets of biomedical KGs can be manually filled out by experts or automatically extracted from Electronic Medical Records (EMRs) and literature . The former is labor-in

In [10]:
top_10 = [x['entity'] for x in results]

Now, getting Llama loaded in here ..

In [11]:
import requests
import json

url = "http://localhost:11434/api/chat"
model = "llama3.1"

In [12]:
restrictions = '''
Only use academic papers from pubmed or biorxiv.
Always cite your sources. 
'''

role = f"""
You are a Biological and Biomedical Knowledge graph LLM. You are a cautious assistant
proficient in creating knowledge graphs for biological and biomedical use cases. 
You are able to find answers to the questions from the contextual passage snippets provided and their affiliated pubmed articles.
Please check the context information carefully and do not use information that is not relevant to the question.
If the retrieved context doesn't provide useful information to answer the question, say that you don't know.
"""

this = """
Use the following pieces of information enclosed in <context> tags sourced from pubmed ids enclosed in <pmid> to provide an answer to the question enclosed in <question> tags.
"""

In [13]:
for each in top_10:
    this+='<context>'
    this+=each['text']
    this+='</context>'

    this+='<pmid>'
    this+=str(each['pmid'])
    this+='</pmid>\n'

this+='<question>'
this+=vector_query
this+='<\question>'
this+=f'''
Here are some restrictions: {restrictions}
'''
print(this)


Use the following pieces of information enclosed in <context> tags sourced from pubmed ids enclosed in <pmid> to provide an answer to the question enclosed in <question> tags.
<context>Knowledge Graph (KG) is a way to store knowledge and reveal the dynamic development law of a field . KG represents facts in the real-world through a large number of triplets (head entity, relation and tail entity), denoted as . Large integrated KGs like Freebase and DBpedia keep expanding . They have been successfully used in many applications, such as recommendation systems and question answering . In the field of biomedicine, the application of KG is becoming increasingly popular for its specialized knowledge that only domain experts can understand well . The influential roles of KG in predicting protein drug targets and adverse drug reactions are both convincing examples . The triplets of biomedical KGs can be manually filled out by experts or automatically extracted from Electronic Medical Records (

In [14]:
system_prompt = {
    'role':'system',
    'content':role
}

user_prompt = {
    'role':'user',
    'content':this
}

promptt = [system_prompt, user_prompt]

In [15]:
def llama3(prompt:list, model='llama3.1'):
    data = {
        "model": model,
        "messages": prompt,
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()["message"]["content"]


In [16]:
response = llama3(promptt)

In [18]:
print(response)

A knowledge graph (KG) is a database that represents structured information as interconnected entities, properties, and relationships in the form of nodes and edges. It is a type of graph database that allows for efficient querying and reasoning about complex data [1].

In the context of biomedical research, KGs can be used to represent a wide range of information, including genomic data, protein structures, disease ontologies, and more [2]. They provide a powerful tool for integrating and analyzing large-scale datasets, and have been used in various applications such as drug discovery, disease modeling, and personalized medicine [3].

KGs can be built using various techniques, including manual curation, semi-automated methods, and automated extraction from text data [4]. They can also be represented in different formats, such as property graphs or RDF triples [5].

References:

[1] Bhole, R., et al. (2020). Knowledge Graphs: A Survey of Methods and Applications. IEEE Access, 8, 127924

In [142]:
# ok = dict(
#     role='user',
#     content='what is a biological knowledge graph useful for? In less than 150 words.'
# )
# llama3(prompt=[ok])
